In [1]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import pandas as pd
import geopandas as gpd
from keplergl import KeplerGl
from tqdm.auto import tqdm
from src.settings import *
from src.tools.osmnx_utils import get_place_dir_name
from src.tools.vis_utils import visualize_kepler, save_config
tqdm.pandas()

In [3]:
cities = pd.read_csv(RAW_DATA_DIR / "cities.csv")
# cities = cities[cities.country == "Poland"]
cities = cities[cities.kacper]
cities

,city,country,continent,kacper,szymon,piotr,kamil,regions,to_fix
1,Nur-Sultan,Kazakhstan,Asia,True,True,False,False,NaN,False
2,Doha,Qatar,Asia,True,False,False,False,NaN,True
3,Moscow,Russia,Asia,True,True,False,True,NaN,False
4,St. Petersburg,Russia,Asia,True,False,False,False,NaN,False
5,Singapore,Singapore,Asia,True,False,False,False,NaN,True
...,...,...,...,...,...,...,...,...,...
107,London,United Kingdom,Europe,True,True,False,True,"['London', 'City of London']",False
112,New York City,United States,North America,True,True,False,False,NaN,True
113,Philadelphia,United States,North America,True,True,False,False,NaN,True
114,San Francisco,United States,North America,True,True,False,False,NaN,True


In [4]:
network_type = "drive"

pbar = tqdm(cities.itertuples(), total=cities.shape[0])
places = []
for row in pbar:
    place_name = f"{row.city},{row.country}"
    place_dir_name = get_place_dir_name(place_name)
    pbar.set_description(place_name)

    try:
        place_df = gpd.read_file(GENERATED_DATA_DIR / place_dir_name / f"graph_{network_type}.gpkg", layer="place")
        places.append(place_df)
    except:
        print("\nFailed", place_name)
    

df_places = pd.concat(places, ignore_index=True)

Sydney,Australia: 100%|██████████| 97/97 [00:11<00:00,  8.27it/s]               


In [5]:
df_places

,level_0,level_1,bbox_north,bbox_south,bbox_east,bbox_west,place_id,osm_type,osm_id,lat,lon,display_name,class,type,importance,area,city,country,continent,geometry
0,0,0,51.351110,51.000677,71.785191,71.224441,282613244,relation,3087155,51.128220,71.430668,"Nur-Sultan, Kazakhstan",boundary,administrative,0.897256,0.089703,Nur-Sultan,Kazakhstan,Asia,"POLYGON ((71.22444 51.10016, 71.24684 51.06827..."
1,0,1,25.392970,25.150541,51.647085,51.395773,282126579,relation,27332,25.285633,51.526416,"Doha, Qatar",boundary,administrative,0.857864,0.014959,Doha,Qatar,Asia,"POLYGON ((51.44787 25.36470, 51.45084 25.35869..."
2,0,2,55.957772,55.491308,37.967428,37.290502,282545482,relation,2555133,55.750446,37.617494,"Moscow, Central Federal District, Russia",place,city,1.010819,0.145873,Moscow,Russia,Asia,"POLYGON ((37.32623 55.68276, 37.32916 55.68192..."
3,0,0,60.090737,59.744148,30.567166,30.043343,281998162,relation,421007,59.917857,30.380619,"Saint Petersburg, Northwestern Federal Distric...",place,city,1.033213,0.095307,St. Petersburg,Russia,Asia,"POLYGON ((30.04334 59.76418, 30.04382 59.76239..."
4,0,0,1.513675,1.126371,104.572181,103.564146,282153901,relation,536780,1.357107,103.819499,Singapore,boundary,administrative,0.963277,0.115579,Singapore,Singapore,Asia,"POLYGON ((103.56415 1.19511, 103.57333 1.19475..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,0,0,51.691874,51.286760,0.334016,-0.510375,281754205,relation,65606,51.489334,-0.144055,"London, Greater London, England, United Kingdom",boundary,administrative,0.940783,0.206480,London,United Kingdom,Europe,"POLYGON ((-0.51038 51.46809, -0.51035 51.46822..."
93,0,0,40.916179,40.477399,-73.700181,-74.259090,281885035,relation,175905,40.712728,-74.006015,"New York, United States",boundary,administrative,1.217577,0.129357,New York City,United States,North America,"POLYGON ((-74.25909 40.49721, -74.25903 40.497..."
94,0,0,40.137959,39.867005,-74.955831,-75.280298,282310523,relation,188022,39.952724,-75.163526,"Philadelphia, Philadelphia County, Pennsylvani...",boundary,administrative,1.023797,0.038974,Philadelphia,United States,North America,"POLYGON ((-75.28030 39.97500, -75.28022 39.974..."
95,0,1,37.929811,37.640314,-122.281479,-123.173825,281816749,relation,111968,37.779026,-122.419906,"San Francisco, California, United States",boundary,administrative,1.125131,0.036107,San Francisco,United States,North America,"POLYGON ((-122.61644 37.81548, -122.61394 37.8..."


In [7]:
config_name = "cities"
m = visualize_kepler({"data": df_places}, config_name=config_name)
m

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 'iuc031q', 'type': …

In [ ]:
save_config(m, config_name=config_name)

WindowsPath('D:/Projekty/Programowanie/Studia/Semestr10/gis/osm-road-infrastructure/assets/keplergl/config/cities.json')

In [ ]:
df_places.dtypes

level_0            int64
level_1            int64
bbox_north       float64
bbox_south       float64
bbox_east        float64
bbox_west        float64
place_id           int64
osm_type          object
osm_id             int64
lat              float64
lon              float64
display_name      object
class             object
type              object
importance       float64
area             float64
city              object
country           object
continent         object
geometry        geometry
dtype: object